In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 9 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Equation de la chaleur - Vision EDP ordre et convergence
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Equation de la chaleur - Vision EDP ordre et convergence

In [ ]:
from dataclasses import dataclass
import numpy as np
from scipy.integrate import solve_ivp
from scipy.sparse import diags
from scipy import sparse
from scipy.sparse.linalg import spsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Equation de la chaleur

On considère l'équation de la chaleur dans le cas 1D :

$$
\left\{
\begin{aligned}
& \partial_t u(x,t) - D \, \partial_{xx} u(x,t) = 0 \;\; \text{pour} \; -L < x < L\\
& u(x,0)  =  \delta_0(x),
\end{aligned}
\right.
$$

où $\delta_0(x)$ désigne la fonction delta de Dirac à l'origine en $x=0$. On prend $L=5$ et $D=1$ et des conditions aux limites de type Neumann homogène.

La solution analytique de l'équation élémentaire de la chaleur sur l'ensemble de la droite réelle s'écrit :

\begin{equation}
u(x,t) = \frac{1}{2 \sqrt{\pi D t}} \exp \Bigg(-\frac{x^2}{\sqrt{4Dt}} \Bigg)
\end{equation}

Les études d'ordre qui suivent sont menées en comparant la solution numérique (pour laquelle on fait tendre à la fois le pas d'espace et de temps vers zéro - au sens des EDP) à la restriction de la solution exacte sur l'intervalle $[-L,L]$, où $L$ a été pris suffisamment grand pour que l'influence des conditions aux limites soit négligeable.

In [ ]:
#############################################
class heat_model:

    def __init__(self, d, xmin, xmax, nx) :
        self.d = d
        self.xmin = xmin
        self.xmax = xmax
        self.nx = nx
        self.dx = (xmax-xmin)/(nx-1)
        self.oneoverdxdx = 1/(self.dx*self.dx)
        
        doverdxdx = d/(self.dx**2)
        diag = np.repeat(-2, nx)
        sub_diag = np.repeat([1., 2.], [nx-2, 1])
        self.a = doverdxdx * diags([diag, sub_diag, np.flip(sub_diag)], [0, -1, 1])

    def fcn(self, t, y):
        return self.a.dot(y)
    
    def fcn_exact(self, t):
        xmin = self.xmin
        xmax = self.xmax
        d = self.d
        nx = self.nx
        dx = self.dx
        x = np.linspace(xmin, xmax, nx)
        y = (1/(2*np.sqrt(np.pi*t))) * np.exp(-(x*x)/(4.*t))
        return y

## Méthode d'Euler explicite

Dans ce cas, la limité de stabilité de la méthode impose $\Delta t \le \frac{1}{2}(\Delta x)^2$.

Pour limiter le coût calcul nous prenons donc exactement une limite où le pas d'espace tend vers zéro avec 
$\Delta t = \frac{1}{2}(\Delta x)^2$.

On peut représenter l'erreur en fonction du pas d'espace (ordre deux) ou en fonction du pas de temps (ordre 1) mais c'est ici le pas d'espace qui pilote en général l'erreur dont la partie spatiale et temporelle sont du même ordre de grandeur.

In [ ]:
@dataclass
class ode_result:
    t: np.ndarray 
    y: np.ndarray    
    
#############################################
def forward_euler(tini, tend, nt, yini, fcn):
    
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    for it, tn  in enumerate(t[:-1]):
        yn = yn + dt*fcn(tn, yn)
        
    return ode_result(t, yn)

In [ ]:
xmin = -5.
xmax = 5.

tini = 0.01
tend = 0.11

d = 1.    

l_dx = []
l_dt = []
l_err_exa = [] 

nx = np.array([1001, 2001, 4001, 8001, 10001, 16001])

for nxi in nx:

    dx = (xmax-xmin)/(nxi-1)

    hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nxi)
    fcn_exact  = hm.fcn_exact
    fcn = hm.fcn

    # initial solution 
    yini = hm.fcn_exact(tini)

    # exact solution 
    yexa = hm.fcn_exact(tend)

    dt = (dx**2)/2
    nt = int((tend-tini)/dt) + 1

    # forward euler integration
    sol = forward_euler(tini, tend, nt, yini, fcn)
    ysol = sol.y

    # error
    err_exa = np.abs(yexa-ysol)
    norm_err_exa = np.linalg.norm(err_exa) / np.sqrt(nxi)
    ##print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_exa:.10e}")

    l_dx.append(dx)
    l_dt.append(dt)
    l_err_exa.append(norm_err_exa)

fig_dx = go.Figure()
fig_dx.add_trace(go.Scatter(x=l_dx, y=l_err_exa, name='erreur'))
fig_dx.add_trace(go.Scatter(x=l_dx, y=np.array(l_dx)**2, name='pente 2'))
fig_dx.update_xaxes(type='log', exponentformat='e', title='dx')
fig_dx.update_yaxes(type='log', exponentformat='e', title='erreur')
fig_dx.update_layout(height=500, title='Erreur EDP : dt = 1/2(dx)^2')
fig_dx.show()

fig_dt = go.Figure()
fig_dt.add_trace(go.Scatter(x=l_dt, y=l_err_exa, name='erreur'))
fig_dt.add_trace(go.Scatter(x=l_dt, y=l_dt, name='pente 1'))
fig_dt.update_xaxes(type='log', exponentformat='e', title='dt')
fig_dt.update_yaxes(type='log', exponentformat='e', title='erreur')
fig_dt.update_layout(height=500, title='Erreur EDP')
fig_dt.show()

## Méthode de Crank-Nicolson

Dans ce cas, comme on n'a pas de limite de stabilité, nous prenons donc deux limites où le pas d'espace tend vers zéro avec $\Delta t = (\Delta x)^2$ (l'erreur temporelle est alors négligeable par rapport à l'erreur spatiale) et on retrouve les résultats précédents (ordre deux en espace et un en temps) mais aussi avec  $\Delta t = \frac{1}{2}\Delta x$ ce qui permet d'avoir deux erreur de même ordre de grandeur pour l'erreur spatiale et temporelle. Dans ce cas,
on obtient un ordre deux en espace et deux en temps.

In [ ]:
#############################################
def crank_nicolson(tini, tend, nt, yini, a):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yn = np.copy(yini)

    cn_mat = sparse.eye(yini.size) - 0.5*dt*a

    for it, tn  in enumerate(t[:-1]):
        yn = spsolve(cn_mat.tocsr(), yn + 0.5*dt*a.dot(yn))

    return ode_result(t, yn)

In [ ]:
xmin = -5.
xmax = 5.

tini = 0.01
tend = 0.11

d = 1.    

print("Cas dt = dx.dx\n")

l_dx = []
l_dt = []
l_err_exa = [] 

nx = np.array([299, 599, 999, 1999, 3999, 5999, 7999])
for nxi in nx:

    dx = (xmax-xmin)/(nxi)

    hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nxi)
    fcn_exact  = hm.fcn_exact
    fcn = hm.fcn
    a = hm.a

    # initial solution 
    yini = hm.fcn_exact(tini)

    # exact solution 
    yexa = hm.fcn_exact(tend)

    dt = dx**2 
    nt = int((tend-tini)/dt) + 1

    # crank-nicolson  integration
    sol = crank_nicolson(tini, tend, nt, yini, a)
    ysol = sol.y

    # error
    err_exa = np.abs(yexa-ysol)
    norm_err_exa = np.linalg.norm(err_exa) / np.sqrt(nxi)
    ##print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_exa:.10e}")

    l_dx.append(dx)
    l_dt.append(dt)
    l_err_exa.append(norm_err_exa)
        
print("\nCas dt = dx/3\n")

l_dx2 = []
l_dt2 = []
l_err_exa2 = [] 

nx2 = np.array([599, 999, 1999, 3999, 5999, 7999, 9999, 15999, 19999])
for nxi in nx2:

    dx = (xmax-xmin)/(nxi-1)

    hm = heat_model(d=d, xmin=xmin, xmax=xmax, nx=nxi)
    fcn_exact  = hm.fcn_exact
    fcn = hm.fcn
    a = hm.a

    # initial solution 
    yini = hm.fcn_exact(tini)

    # exact solution 
    yexa = hm.fcn_exact(tend)

    dt = dx / 3
    nt = int((tend-tini)/dt) + 1

    # crank-nicolson  integration
    sol = crank_nicolson(tini, tend, nt, yini, a)
    ysol = sol.y

    # error
    err_exa = np.abs(yexa-ysol)
    norm_err_exa = np.linalg.norm(err_exa) / np.sqrt(nxi)
    ##print(f"Norme l2 de l'erreur pour dt = {dt:.3e} : {norm_err_exa:.10e}")

    l_dx2.append(dx)
    l_dt2.append(dt)
    l_err_exa2.append(norm_err_exa)
        
        
fig_dx = go.Figure()
fig_dx.add_trace(go.Scatter(x=l_dx, y=l_err_exa, name='erreur (dt=dx.dx)'))
fig_dx.add_trace(go.Scatter(x=l_dx, y=np.array(l_dx)**2, name='pente 2'))
fig_dx.add_trace(go.Scatter(x=l_dx2, y=l_err_exa2, name='erreur (dt=dx/3)'))
fig_dx.update_xaxes(type='log', exponentformat='e', title='dx')
fig_dx.update_yaxes(type='log', exponentformat='e', title='erreur')
fig_dx.update_layout(height=500, title='Erreur EDP')
fig_dx.show()

fig_dt = go.Figure()
fig_dt.add_trace(go.Scatter(x=l_dt, y=l_err_exa, name='erreur (dt=dx.dx)'))
fig_dt.add_trace(go.Scatter(x=l_dt, y=l_dt, name='pente 1'))
fig_dt.add_trace(go.Scatter(x=l_dt2, y=l_err_exa2, name='erreur (dt=dx/3)'))
fig_dt.add_trace(go.Scatter(x=l_dt2, y=np.array(l_dt2)**2, name='pente 2', ))

fig_dt.update_xaxes(type='log', exponentformat='e', title='dt')
fig_dt.update_yaxes(type='log', exponentformat='e', title='erreur')
fig_dt.update_layout(height=500, title='Erreur EDP')
fig_dt.show()